### Importing the libraries

In [1]:
from keras.models import load_model

mod=load_model()

Using TensorFlow backend.


In [11]:
mod

In [1]:
#Imports for pre-processing of images and other utlity libraries
import random
import os
import shutil
from tqdm import tqdm
import matplotlib.pyplot as plt
import imageio
from PIL import Image
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix , classification_report , accuracy_score

#Import the deep learning libraries
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array,array_to_img
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.models import load_model
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import optimizers
from keras.models import Model
from keras.callbacks import History
from keras.applications import vgg16, vgg19, inception_v3, xception, resnet, inception_v3, inception_resnet_v2, mobilenet, mobilenet, nasnet, mobilenet_v2

Using TensorFlow backend.


### Utility functions

In [2]:
def plt_epoch_error(x, val_error, train_error, colors=['b']):
    """
    This function is used to plot the loss vs epoch for the
    trained models using the History callback.
    """
    plt.figure(figsize=(12,6))
    plt.grid()
    plt.plot(x, val_error, 'b', label="Validation Loss")
    plt.plot(x, train_error, 'r', label="Train Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    
def plot_confusion_matrix(test_y, predict_y):
    """
    Given the ground truth and the predicted labels,
    this function is used to plot the confusion matrix,
    recall matrix and the precision matrix using seaborn
    heatmaps.
    """
    C = confusion_matrix(test_y, predict_y) #Confusion Matrix
    A =(((C.T)/(C.sum(axis=1))).T) #Recall Matrix
    B =(C/C.sum(axis=0)) #Precision Matrix
    
    plt.figure(figsize=(20,4))
    labels = [0,1]
    
    #Representing Confusion Matrix(C) in heatmap format
    cmap=sns.light_palette("blue")
    plt.subplot(1, 3, 1)
    sns.heatmap(C, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Confusion matrix")
    
    #Representing Precision Matrix(B) in heatmap format
    plt.subplot(1, 3, 2)
    sns.heatmap(B, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Precision matrix")
    
    plt.subplot(1, 3, 3)
    #Representing Recall Matrix(B) in heatmap format
    sns.heatmap(A, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Recall matrix")
    
    plt.show()

In [5]:
filepath="""weights\"""
filepath

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-5-6a479c944b0f>, line 2)

### Model

In [ ]:
!wget https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5 --no-check-certificate
!wget https://github.com/fchollet/deep-learning-models/releases/download/v0.5/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 --no-check-certificate
!wget https://github.com/fchollet/deep-learning-models/releases/download/v0.7/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5 --no-check-certificate
!wget https://github.com/titu1994/Keras-NASNet/releases/download/v1.2/NASNet-large-no-top.h5 --no-check-certificate
!wget https://github.com/keras-team/keras-applications/releases/download/resnet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5 --no-check-certificate
!wget https://github.com/fchollet/deep-learning-models/releases/download/v0.4/xception_weights_tf_dim_ordering_tf_kernels.h5 --no-check-certificate              

In [11]:
filepath="C:\\Users\\Hp\\Documents\\e-Crash-master\\weights\\"
os.listdir(filepath)

base_model = inception_v3.InceptionV3(weights=None, include_top=False)
base_model.load_weights(filepath+"inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5")

In [12]:
base_model=load_models("inceptionv3")
#Adding a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

In [20]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

import pandas as pd
def load_data():
    df_train=pd.read_csv("train.csv")
    df_test=pd.read_csv("test.csv")
    df_val=pd.read_csv("val.csv")
    return df_train, df_test, df_val

#Creating the base pre-trained model (VGG16)
#https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5
def load_models(name):
    """
    Initialize the pre-trained model architecture and load the model weights.
    The downloaded weights contains only the convolution base. It does not 
    contain the top two dense layers. We will have to manually define the top
    two dense layers.
    """
    filepath="C:\\Users\\Hp\\Documents\\e-Crash-master\\weights\\"
    if(name=='vgg16'):
        base_model = vgg16.VGG16(weights=None, include_top=False)
        base_model.load_weights(filepath+"vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5")
    elif(name=="inceptionv3"):
        base_model = inception_v3.InceptionV3(weights=None, include_top=False)
        base_model.load_weights(filepath+"inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5")
    elif(name=="resnet50"):
        base_model = resnet.ResNet50(weights=None, include_top=False)
        base_model.load_weights(filepath+"resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5")
    elif(name=="inception_resnet"):
        base_model = inception_resnet_v2.InceptionResNetV2(weights=None, include_top=False)
        base_model.load_weights(filepath+"inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5")        
    elif(name=="nasnet"):
        base_model = nasnet.NASNetLarge(weights=None, include_top=False)
        base_model.load_weights(filepath+"NASNet-large-no-top.h5")   
    elif(name=="xception"):
        base_model = xception.Xception(weights=None, include_top=False)
        base_model.load_weights(filepath+"inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5")      
    return base_model

#Define custom callbacks
def callbacks_list(monitor):
    filepath="C:\\Users\\Hp\\Documents\\e-Crash-master\\models\\"
    checkpoint = ModelCheckpoint(filepath+"weights-{epoch:02d}-{val_acc:.2f}.hdf5", 
                                 monitor=monitor, 
                                 verbose=1, 
                                 save_best_only=False, 
                                 mode='auto',
                                 period=1)

    reduce_learning_rate = ReduceLROnPlateau(monitor=monitor, patience=10)
    early_stop = EarlyStopping(monitor=monitor, patience=10)
    history = History()
    list_ = [checkpoint, reduce_learning_rate, early_stop, history]
    return list_

def train_stage1(model_name, dense_neurons, classes, batch_size, epochs, stage1_lr):
    base_model = load_models(model_name)
    #Adding a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    #Adding a fully-connected dense layer
    x = Dense(dense_neurons, activation='relu', kernel_initializer='he_normal')(x)

    #Adding a final dense output final layer
    predictions = Dense(classes, activation='softmax', kernel_initializer='glorot_uniform')(x)

    #Define the model
    model_stg1 = Model(inputs=base_model.input, outputs=predictions)

    #Here we will freeze the convolution base and train only the top layers
    #We will set all the convolution layers to false, the model should be
    #compiled when all the convolution layers are set to false
    for layer in base_model.layers:
        layer.trainable = False

    #Compiling the model
    model_stg1.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    
    #Normalize the images
    train_datagen = ImageDataGenerator(rescale=1./255)
    val_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)
                  
    df_train, df_test, df_val = load_data()
    
    delim='\\'
    source="data_split"

    train_generator = train_datagen.flow_from_dataframe(dataframe=df_train,
                                                        directory=os.path.abspath(source+"{}train".format(delim)),
                                                        target_size=(224, 224),
                                                        x_col="filenames",
                                                        y_col="class_label",                                                    
                                                        batch_size=batch_size,
                                                        class_mode='categorical')

    val_generator = val_datagen.flow_from_dataframe(dataframe=df_val,
                                                    directory=os.path.abspath(source+"{}validation".format(delim)),
                                                    target_size=(224, 224),
                                                    x_col="filenames",
                                                    y_col="class_label",                                                    
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

    test_generator = test_datagen.flow_from_dataframe(dataframe=df_test,
                                                      directory=os.path.abspath(source+"{}test".format(delim)),
                                                      target_size=(224, 224),
                                                      x_col="filenames",
                                                      y_col="class_label",                                                    
                                                      batch_size=batch_size,
                                                      class_mode='categorical')


    nb_train_samples = len(train_generator.classes)
    nb_val_samples = len(val_generator.classes)
    nb_test_samples = len(test_generator.classes)

    model_stg1.fit_generator(train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=epochs,
                        validation_data=val_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=callbacks_list())
                  
    params=[train_generator,
            val_generator,
            test_generator,
            nb_train_samples,
            nb_val_samples,
            nb_test_samples, 
            batch_size]
    
    return model_stg1, params

def train_stage2(stage2_lr, model_stg2, model_name, epochs):
    #At this point, the top layers are well trained and we can start fine-tuning
    #convolutional layers of the pre-trained architecture. We will freeze the bottom 
    #N layers and train the remaining top layers.

    #We will train the top 2 blocks and we will freeze the first N layers
    #A very low learning rate is used, so as to not wreck the convolution base with 
    #massive gradient updates
                         
    if(model_name=='vgg16'):
        for layer in model_stg2.layers[:15]:
           layer.trainable = False
        for layer in model_stg2.layers[15:]:
           layer.trainable = True
    elif(name=="inceptionv3"):
        for layer in model_stg2.layers[:15]:
           layer.trainable = False
        for layer in model_stg2.layers[15:]:
           layer.trainable = True
    elif(name=="resnet50"):
        for layer in model_stg2.layers[:15]:
           layer.trainable = False
        for layer in model_stg2.layers[15:]:
           layer.trainable = True
    elif(name=="inception_resnet"):
        for layer in model_stg2.layers[:15]:
           layer.trainable = False
        for layer in model_stg2.layers[15:]:
           layer.trainable = True        
    elif(name=="nasnet"):
        for layer in model_stg2.layers[:15]:
           layer.trainable = False
        for layer in model_stg2.layers[15:]:
           layer.trainable = True   
    elif(name=="xception"):
        for layer in model_stg2.layers[:15]:
           layer.trainable = False
        for layer in model_stg2.layers[15:]:
           layer.trainable = True      

    #Recompile the model, train the top 2 blocks
    model_stg2.compile(optimizer=optimizers.RMSprop(lr=stage2_lr), 
                       loss='categorical_crossentropy', 
                       metrics=['accuracy'])

    #Re-train the model again (this time fine-tuning the top 2 inception blocks
    #alongside the top Dense layers
    model_stg2.fit_generator(params[0],
                             steps_per_epoch=nb_train_samples // params[6],
                             epochs=epochs,
                             validation_data=params[1],
                             validation_steps=params[4] // params[6],
                             callbacks=callbacks_list())
    return model_stg2
                  
def train(model_name, dense_neurons, classes, batch_size, epochs,stage1_lr,stage2_lr):
    model_stg1, params = train_stage1(model_name, dense_neurons, classes, batch_size, epochs, stage1_lr)
    model_stg2 = train_stage2(stage2_lr, model_stg1, model_name, epochs)
    
    return model_stg1, model_stg2
    


In [21]:
train("inceptionv3", dense_neurons=128, classes=2, batch_size=5, epochs=2,stage1_lr=0.00001,stage2_lr=0.00001)

KeyboardInterrupt: 

In [ ]:
delim='\\'
source="data_split"
os.path.abspath(source+"{}train".format(delim))

In [ ]:
df_train

In [ ]:
filepath=os.path.abspath(source+"{}".format(delim))
filepath="'C:\\Users\\Hp\\Documents\\e-Crash-master\\models"

In [ ]:
#Define custom callbacks
def callbacks_list():
    filepath=os.path.abspath(source+"{}".format(delim))
    checkpoint = ModelCheckpoint("weights-{epoch:02d}-{val_acc:.2f}.hdf5", 
                                 monitor='val_acc', 
                                 verbose=1, 
                                 save_best_only=False, 
                                 mode='max',
                                 period=1)

    reduce_learning_rate = ReduceLROnPlateau(monitor='val_acc', patience=10)
    early_stop = EarlyStopping(monitor='val_acc', patience=10)
    history = History()
    list_ = [checkpoint, reduce_learning_rate, early_stop, history]
    return list_

list_=callbacks_list()
list_
    

delim='\\'
source="data_split"
os.path.abspath(source+"{}train".format(delim))

#Training the model on sample dataset
#Normalize the images

batch_size = 5
epochs = 2

train_datagen = ImageDataGenerator(rescale=1./255)

#Normalize the images
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(dataframe=df_train,
                                                    directory=os.path.abspath(source+"{}train".format(delim)),
                                                    target_size=(224, 224),
                                                    x_col="filenames",
                                                    y_col="class_label",                                                    
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

val_generator = val_datagen.flow_from_dataframe(dataframe=df_val,
                                                    directory=os.path.abspath(source+"{}validation".format(delim)),
                                                    target_size=(224, 224),
                                                    x_col="filenames",
                                                    y_col="class_label",                                                    
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

test_generator = test_datagen.flow_from_dataframe(dataframe=df_test,
                                                    directory=os.path.abspath(source+"{}test".format(delim)),
                                                    target_size=(224, 224),
                                                    x_col="filenames",
                                                    y_col="class_label",                                                    
                                                    batch_size=batch_size,
                                                    class_mode='categorical')


nb_train_samples = len(train_generator.classes)
nb_val_samples = len(val_generator.classes)
nb_test_samples = len(test_generator.classes)

model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=epochs,
                    validation_data=val_generator,
                    validation_steps=nb_val_samples // batch_size,
                    callbacks=callbacks_list())

In [ ]:
for i, layer in enumerate(model.layers):
   print(i, layer.name)

In [ ]:
#Load the best weights and save the final model
model.load_weights("vgg19_weights.hdf5")
model.save("vgg19_model.h5")

In [ ]:
#At this point, the top layers are well trained and we can start fine-tuning
#convolutional layers of the pre-trained architecture. We will freeze the bottom 
#N layers and train the remaining top layers.

#Visualize the layer names and indices to see how many layers do we need to train
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

#We will train the top 2 blocks and we will freeze the first N layers
#A very low learning rate is used, so as to not wreck the convolution base with 
#massive gradient updates
for layer in model.layers[:15]:
   layer.trainable = False
for layer in model.layers[15:]:
   layer.trainable = True

#Recompile the model, train the top 2 blocks
from keras.optimizers import SGD
model.compile(optimizer=RMSprop(lr=0.000001), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#Re-train the model again (this time fine-tuning the top 2 inception blocks
#alongside the top Dense layers
model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=10,
                    validation_data=val_generator,
                    validation_steps=nb_val_samples // batch_size)

In [ ]:
history=model.history

#List of epoch numbers
x = list(range(0,len(history.history['val_loss'])))

#Plot train vs test loss
val_error = history.history['val_loss'] 
train_error = history.history['loss'] 
plt_epoch_error(x, val_error, train_error)

In [ ]:
len(Y_pred)

In [ ]:
#Confution Matrix and Classification Report
Y_pred = model.predict_generator(validation_generator, nb_validation_samples // batch_size)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['Handwritten','Printed']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))